In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])
if manual:
    %matplotlib inline


Bad key text.latex.preview in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
from tqdm import tqdm
from quetzal.model import stepmodel
from quetzal.engine import engine
from quetzal.io import excel

# Preparation of the transport network.
## Replaces scheduled bus network with on-demand services.
## Needs clustered PT network and connectors.

In [3]:
input_path = '../input/'
output_path = '../output/'
model_path = '../model/' + scenario + '/'

In [71]:
# Load scenario parameters
params = excel.read_var(file='../input/parameters.xls', scenario=scenario)

In [43]:
# Loading StepModel with PT network...
sm = stepmodel.read_json(model_path + 'de_pt_network_agg')
ae = stepmodel.read_json(model_path + 'de_pt_access_egress')
sm.centroids = ae.centroids
sm.footpaths = ae.footpaths
sm.zone_to_transit = ae.zone_to_transit

In [19]:
len(sm.nodes)

36462

In [45]:
# Drop scheduled bus services outside of cities
drop_nodes = sm.nodes.loc[(sm.nodes['route_type']=='bus')
                          & (sm.nodes['FID'].map(sm.zones['urbanisation'])!=1)
                         ].index
# Keep nodes from trips to/from cities
city_trips = set(sm.links.loc[~sm.links['a'].isin(drop_nodes), 'trip_id']).union(
             set(sm.links.loc[~sm.links['b'].isin(drop_nodes), 'trip_id']))
drop_nodes = set(drop_nodes) \
             - set(sm.links.loc[sm.links['trip_id'].isin(city_trips), 'a']) \
             - set(sm.links.loc[sm.links['trip_id'].isin(city_trips), 'b'])
# Drop
sm.footpaths = sm.footpaths.loc[~((sm.footpaths['a'].isin(drop_nodes))
                                  | (sm.footpaths['b'].isin(drop_nodes)))]
sm.zone_to_transit = sm.zone_to_transit.loc[~((sm.zone_to_transit['a'].isin(drop_nodes))
                                              | (sm.zone_to_transit['b'].isin(drop_nodes)))]
sm.links = sm.links.loc[~((sm.links['a'].isin(drop_nodes))
                          | (sm.links['b'].isin(drop_nodes)))]
sm.nodes = sm.nodes.drop(drop_nodes)

In [61]:
# Drop broken trip sequences
def test_sequences_save(trip):
    if len(trip)!=trip['link_sequence'].max():
        return list(trip.index)
broken = sm.links.loc[(sm.links['route_type']=='bus')
                     ].groupby('trip_id').apply(test_sequences_save)
sm.links.drop(broken.loc[broken.notna()].sum(), inplace=True)

In [ ]:
# Plot the coverage of scheduled bus lines
if manual:
    sm.links = gpd.GeoDataFrame(sm.links)
    sm.zones = gpd.GeoDataFrame(sm.zones)
    sm.links.loc[sm.links['route_type']=='bus', 'geometry'].plot(
        ax=sm.zones.loc[sm.zones['urbanisation']==1].plot(color='green', alpha=.3, figsize=(5,8)))

In [63]:
len(sm.nodes)

20084

In [9]:
sm.links.sample()

,route_type,geometry,a,b,link_sequence,route_id,time,trip_id,headway,route_type_disagg
index,,,,,,,,,,
rs_23269,rail_short,"LINESTRING (8.42302 49.00355, 8.41651 49.00498)",rs_DE122_0_35,rs_DE122_0_24,3,rs_204,60,rs_204_2,21600,rail_short


In [10]:
sm.nodes.sample()

,FID,route_type,geometry
index,,,
rs_DE94B_2_0,DE94B_2,rail_short,POINT (7.07477 52.41700)


## Build on-demand services

One service to every other PT node and centroid within a distance threshold from each zone's centroid.

In [ ]:
def links_nodes_footpaths_connectors_from_ntlegs(ntlegs):
    # Build links
    links = ntlegs[['time', 'geometry']]
    links['route_type'] = 'bus'
    links['route_type_disagg'] = 'on-demand'
    links['headway'] = 0
    links['trip_id'] = 
    links['link_sequence'] = 1
    links['trip_id'] = ntlegs['a'] + '-' + ntlegs['b']
    links['route_id'] = ntlegs['a']
    links['a'] = 'on-dem_' + ntlegs['a']
    links['b'] = 'on-dem_' + ntlegs['b']
    links.index = 'on-dem_' + ntlegs.reset_index().index.apply(str)
    # Build nodes
    nodes = ntlegs[['a']]
    nodes['geometry'] = ntlegs['geometry'].apply(lambda l: l.coords[0][0])
    nodes['FID'] = ntlegs.loc[ntlegs['direction']=='access', 'a'].append(
        ntlegs.loc[ntlegs['direction']=='eggress', 'a'].map(sm.nodes['FID']))
    nodes['route_type'] = 'bus'
    nodes.index = 'on-dem_' + ntlegs['a']
    # Build footpaths
    foot = pd.DataFrame()
    #foot = ntlegs.loc[ntlegs['direction']=='access', ['b', 'direction']]
    #foot['direction']
    #foot['a'] = 'on-dem_' + ntlegs.loc[ntlegs['direction']=='access', 'a']
    foot['distance'] = 0
    foot['time'] = 0
    foot['speed'] = params['pt_access']['speed_walk']
    foot['geometry'] = 
    # Build centroid connectors (zone_to_transit)
    

In [ ]:
# Build services to PT nodes
mask = sm.zones['urbanisation']!=1
ntlegs = engine.ntlegs_from_centroids_and_nodes(
    sm.centroids.head(),
    sm.nodes.loc[sm.nodes['route_type']!='bus'],
    n_neighbors=50, # double that times number of zones will be added to links
    coordinates_unit=sm.coordinates_unit)
ntlegs = ntlegs.loc[ntlegs['distance']<=params['on-demand_bus']['max_dist_pt']]
ntlegs['time'] = ntlegs['distance']/1000 * params['on-demand_bus']['speed'] / 3600
len(ntlegs)

In [ ]:
# Every zone should have an access and an egress link to PT
assert sm.zones['FID'].isin(list(sm.zone_to_transit['a'])).all()
assert sm.zones['FID'].isin(list(sm.zone_to_transit['b'])).all()

### Parametrise access and egress  links

Add distance, speed and time. Average distances by zone and mode should have been calculated based on census data before.

## Save model
And reduce the file size

In [61]:
try:
    sm.integrity_test_nodeset_consistency()
except AssertionError:
    print('Number of orphan nodes: {}'.format(
        len(sm.orphan_nodes)))
    print('Number of missing nodes: {}'.format(
        len(sm.missing_nodes)))

Number of orphan nodes: 68
Number of missing nodes: 0


In [62]:
assert len(sm.missing_nodes) == 0
if len(sm.orphan_nodes) > 0:
    sm.nodes = sm.nodes.drop(sm.orphan_nodes)
    sm.nodes.shape

In [65]:
# Cast columns to int
cols = ['time', 'link_sequence', 'headway']
sm.links[cols] = sm.links[cols].astype(int)

In [66]:
# Split links in graph and auxiliary information
# for file sizes being compatible with github's size limit
cols = ['a', 'b', 'link_sequence', 'route_id', 'time', 'trip_id', 'headway', 'route_type_disagg']
auxiliary = sm.links[cols]
sm.links.drop(cols, axis=1, inplace=True)

In [67]:
# Saving model...
sm.to_json(model_path + 'de_pt_network_agg',
           only_attributes=['zones', 'links', 'nodes', 'pt_route_types'],
           encoding='utf-8')

to_hdf(overwriting): 100%|█████████████████████████████████████████████████████████████████████| 36/36 [00:00<?, ?it/s]


In [68]:
# Save auxiliary information seperately
auxiliary['index'] = auxiliary.index
auxiliary.reset_index(drop=True, inplace=True)
auxiliary.to_json(model_path + 'de_pt_network_agg/links_quetzaldata.json')

In [69]:
# Save reduced access/egress links
sm.to_json(model_path + 'de_pt_access_egress', only_attributes=[
    'centroids', 'footpaths', 'zone_to_transit'], encoding='utf-8')

to_hdf(overwriting): 100%|█████████████████████████████████████████████████████████████| 36/36 [00:10<00:00,  3.31it/s]
